In [ ]:
import matplotlib.pyplot as plt
import numpy.random as rnd
import numpy as np

%matplotlib notebook


In [ ]:
NUM_ITERATIONS = 500

x1, x2 = [-0.1, 0.7]
samples = []

for i in range(500):
    # Sample x1 given x2, using Pythagorean theorem
    x1_max = np.sqrt(1 - x2**2)
    x1_min = -x1_max
    x1 = np.random.uniform(x1_min, x1_max)
    samples.append([x1, x2])
    
    # Sample x2 given x1 similarly
    x2_max = np.sqrt(1 - x1**2)
    x2_min = -x2_max
    x2 = np.random.uniform(x2_min, x2_max)
    samples.append([x1, x2])
    
samples_arr = np.array(samples)
plt.figure()
plt.scatter(samples_arr[:, 0], samples_arr[:, 1], color='orange')
plt.axis('equal')


### Gibbs sampling

In [ ]:
t = np.arange(0, 2 * np.pi, .001)

x_circ = np.cos(t)
y_circ = np.sin(t)

plt.figure()
plt.plot(x_circ, y_circ, lw=2)
plt.axis('equal')
plt.xlabel('x1')
plt.ylabel('x2')
initial_point = np.array([0.2, -0.3])

x1_val, x2_val = initial_point

plt.scatter(x1_val, x2_val, color='red')
stage = 'find_x1_dist_given_x2'
line = None


In [ ]:
### Interactive demo cell
# Keep running this over and over to generate points on the graph from the
# previous cell. Make sure you're using `%matplotlib notebook`
# Each time you run this cell, the variable `stage` controls which of the
# 4 if/elif blocks executes, and then changes it so that the next block
# will execute next time.

print(stage)
# Draw the horizontal line
if stage == "find_x1_dist_given_x2":
    if line:
        line.remove()
    x1_max = np.sqrt(1 - x2_val ** 2)
    x1_min = -x1_max
    # Draw horizontal line
    line,  = plt.plot([x1_min, x1_max], [x2_val, x2_val], color='black', lw=1)
    stage = 'sample_x1_given_x2'

# Sample a point
elif stage == 'sample_x1_given_x2':
    plt.scatter(x1_val, x2_val, color='gray')
    x1_val = np.random.uniform(x1_min, x1_max)
    plt.scatter(x1_val, x2_val, color='red')
    stage = "find_x2_dist_given_x1"
    
# Draw the vertical line
elif stage == "find_x2_dist_given_x1":
    if line:
        line.remove()
    x2_max = np.sqrt(1 - x1_val ** 2)
    x2_min = -x2_max
    # Draw vertical line
    line,  = plt.plot([x1_val, x1_val], [x2_min, x2_max], color='black', lw=1)
    stage = 'sample_x2_given_x1'

# Sample a point
elif stage == 'sample_x2_given_x1':
    plt.scatter(x1_val, x2_val, color='gray')
    x2_val = np.random.uniform(x2_min, x2_max)
    plt.scatter(x1_val, x2_val, color='red')
    stage = 'find_x1_dist_given_x2'
plt.savefig('7.pdf')

## Metropolis sampling from scratch

In [ ]:
import random
def normal(x,mu,sigma):
    numerator = np.exp((-(x-mu)**2)/(2*sigma**2))
    denominator = sigma * np.sqrt(2*np.pi)
    return numerator/denominator

def random_coin(p):
    unif = random.uniform(0,1)
    if unif>=p:
        return False
    else:
        return True
    
def gaussian_MS(hops,mu,sigma):
    print(hops)
    states = []
    proposal = []
    accept = []
    burn_in = int(hops*0.2)
    current = random.uniform(-5*sigma+mu,5*sigma+mu)
    for i in range(hops):
        states.append(current)
        movement = random.uniform(-5*sigma+mu,5*sigma+mu)
        proposal.append(movement)
        curr_prob = normal(x=current,mu=mu,sigma=sigma)
        move_prob = normal(x=movement,mu=mu,sigma=sigma)
        
        acceptance = min(move_prob/curr_prob,1)
        if random_coin(acceptance):
            current = movement
            accept.append(1)
        else:
            accept.append(0)
    return states, states[burn_in:], accept, proposal
    
lines = np.linspace(-3,3,1000)
normal_curve = [normal(l,mu=0,sigma=1) for l in lines]
allpts, dist, acc, prop = gaussian_MS(100000,mu=0,sigma=1)
plt.figure()
plt.hist(dist,bins=100,density=True,stacked=True) 
plt.plot(lines,normal_curve,'r')
plt.show()

In [ ]:
%matplotlib inline
plt.figure()
plt.plot(allpts)
plt.plot([0.2*100000,0.2*100000],[-4,4],'--',label='Burn-in')
plt.legend()
plt.show()

In [ ]:
#To take a look at a small portion of samples
tmp_prop = np.array(prop[10000:10500])
tmp_acc = np.array(acc[10000:10500])
x = np.arange(500)
plt.figure()
plt.plot(allpts[10000:10500], label="Path")
plt.plot(x[tmp_acc==1],tmp_prop[tmp_acc==1],'g.',label='Accepted',alpha=0.3)
plt.plot(x[tmp_acc==0],tmp_prop[tmp_acc==0], 'rx', label='Rejected',alpha=0.3)
plt.legend()
plt.show()